# Azure AI Search와 NVIDIA NIM 및 LlamaIndex 통합

이 노트북에서는 NVIDIA의 AI 모델과 LlamaIndex를 활용하여 강력한 Retrieval-Augmented Generation (RAG) 파이프라인을 구축하는 방법을 보여줍니다. NVIDIA의 LLM과 임베딩을 사용하고, 이를 Azure AI Search를 벡터 저장소로 통합하여 검색 품질과 효율성을 향상시키는 RAG를 수행할 것입니다.

## 장점
- **확장성**: NVIDIA의 대규모 언어 모델과 Azure AI Search를 활용하여 확장 가능하고 효율적인 검색을 제공합니다.
- **비용 효율성**: 효율적인 벡터 저장소와 하이브리드 검색 기술을 통해 검색 및 검색 최적화.
- **고성능**: 강력한 LLM과 벡터화된 검색을 결합하여 더 빠르고 정확한 응답을 제공합니다.
- **품질**: 관련 문서를 기반으로 LLM 응답의 품질을 유지합니다.

## 사전 준비 사항
- 🐍 Python 3.9 이상
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API 키 (NVIDIA NIM 마이크로서비스를 통해 NVIDIA의 LLM 및 임베딩에 액세스)

## 포함된 기능
- ✅ NVIDIA LLM 통합 (우리는 [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe)를 사용할 예정입니다)
- ✅ NVIDIA 임베딩 (우리는 [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5)를 사용할 예정입니다)
- ✅ Azure AI Search 고급 검색 모드
- ✅ LlamaIndex를 사용한 문서 색인화
- ✅ NVIDIA LLM을 활용한 Azure AI Search와 LlamaIndex를 통한 RAG

시작해봅시다!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## 설치 및 요구 사항
Python 버전 >3.10을 사용하여 Python 환경을 만드세요.

## 시작하기!


NVIDIA AI Foundation 모델을 사용하려면 `NVIDIA_API_KEY`가 필요합니다:
1) [NVIDIA](https://build.nvidia.com/explore/discover)에서 무료 계정을 생성하세요.
2) 원하는 모델을 선택하세요.
3) 입력 섹션에서 Python 탭을 선택한 후 **Get API Key**를 클릭하고 **Generate Key**를 클릭하세요.
4) 생성된 키를 복사하여 NVIDIA_API_KEY로 저장하세요. 그러면 엔드포인트에 접근할 수 있습니다.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## LLM과 임베딩을 활용한 RAG 예제
### 1) LLM 초기화
`llama-index-llms-nvidia`, 즉 NVIDIA의 LLM 커넥터는 NVIDIA API 카탈로그에서 제공되는 호환 가능한 모델에 연결하고 이를 통해 생성할 수 있도록 합니다. 채팅 완료 모델 목록은 다음 링크를 참조하세요: https://build.nvidia.com/search?term=Text-to-Text

여기서는 **mixtral-8x7b-instruct-v0.1**을 사용할 것입니다.


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) 임베딩 초기화
`llama-index-embeddings-nvidia`, 또는 NVIDIA의 Embeddings 커넥터는 NVIDIA API 카탈로그에서 제공되는 호환 가능한 모델에 연결하고 이를 통해 생성할 수 있도록 합니다. 우리는 임베딩 모델로 `nvidia/nv-embedqa-e5-v5`를 선택했습니다. 텍스트 임베딩 모델 목록은 다음을 참조하세요: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) Azure AI 검색 벡터 스토어 생성


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) 데이터에 대한 질문을 할 수 있는 쿼리 엔진 생성하기

다음은 Azure AI Search에서 순수 벡터 검색을 사용하고 우리의 LLM(Phi-3.5-MOE)에 응답을 연결하는 쿼리입니다.


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Azure AI Search에서 하이브리드 검색을 사용하는 쿼리입니다.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### 벡터 검색 분석
LLM 응답은 러시아 주식 시장에 대한 주요 경제적 결과를 정확히 포착하고 있습니다. 특히, 러시아 주식 시장이 40%의 가치 하락을 겪었으며, 현재 상황으로 인해 거래가 중단되었다고 명시하고 있습니다. 이 응답은 출처에서 제공된 정보와 잘 일치하며, 러시아의 행동과 부과된 제재로 인해 주식 시장에 미친 영향을 LLM이 올바르게 식별하고 요약했음을 보여줍니다.

#### 출처 노드 주석
출처 노드는 국제 제재로 인해 러시아가 직면한 경제적 결과를 상세히 설명합니다. 텍스트는 러시아 주식 시장이 40%의 가치를 잃었고 거래가 중단되었다는 점을 강조합니다. 또한 루블화의 평가 절하와 러시아 경제의 전반적인 고립과 같은 다른 경제적 여파도 언급하고 있습니다. LLM 응답은 이러한 노드에서 핵심 사항을 효과적으로 추출하여, 쿼리에서 요청된 대로 주식 시장 영향을 중심으로 요약했습니다.


이제 Hybrid Search가 명확한 답변을 제공하지 못하는 경우를 살펴보겠습니다:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### 하이브리드 검색: LLM 응답 분석
하이브리드 검색 예시에서 LLM 응답은 제공된 문맥이 러시아의 우크라이나 침공 정확한 날짜를 명시하지 않았음을 나타냅니다. 이 응답은 LLM이 소스 문서에 있는 정보를 활용하면서도 텍스트에 정확한 세부 사항이 없음을 인정하고 있음을 보여줍니다.

응답은 문맥이 러시아의 공격과 관련된 사건을 언급하지만 특정 침공 날짜를 명확히 하지 않는다는 점을 정확히 파악하고 있습니다. 이는 LLM이 제공된 정보를 이해하면서도 콘텐츠의 빈틈을 인식하는 능력을 보여줍니다. LLM은 정보가 불완전할 때 사용자가 외부 소스나 역사적 기록을 찾아보도록 효과적으로 유도하며 신중함을 드러냅니다.

### 소스 노드 분석
하이브리드 검색 예시의 소스 노드에는 미국이 러시아의 행동에 대응하는 방식을 논의한 연설의 발췌문이 포함되어 있습니다. 이 노드들은 침공에 대한 미국과 동맹국들의 대응 조치와 더 넓은 지정학적 영향을 강조하지만, 특정 침공 날짜는 언급하지 않습니다. 이는 문맥에 정확한 날짜 정보가 없음을 올바르게 파악한 LLM 응답과 일치합니다.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### 하이브리드 w/재랭킹: LLM 응답 분석
하이브리드 w/재랭킹 예제에서는 LLM 응답이 연설이 이루어진 시점으로부터 사건이 6일 전에 발생했다는 추가적인 맥락을 제공합니다. 이는 LLM이 연설 시점을 기반으로 침공 날짜를 추론할 수 있음을 나타내며, 정확성을 위해 연설의 정확한 날짜를 여전히 알아야 한다는 점을 보여줍니다.

이 응답은 맥락 단서를 활용하여 더 유익한 답변을 제공하는 향상된 능력을 보여줍니다. 이는 재랭킹의 장점을 강조하며, LLM이 더 관련성 높은 정보를 접근하고 우선순위를 매겨 원하는 세부사항(즉, 침공 날짜)에 더 가까운 답변을 제공할 수 있음을 나타냅니다.

### 소스 노드 분석
이 예제의 소스 노드에는 러시아의 침공 시점에 대한 언급이 포함되어 있으며, 침공이 연설 6일 전에 발생했다는 점을 구체적으로 언급합니다. 정확한 날짜는 여전히 명시적으로 제공되지 않지만, 노드들은 LLM이 더 세밀한 응답을 제공할 수 있도록 하는 시간적 맥락을 제공합니다. 이러한 세부사항의 포함은 재랭킹이 제공된 맥락에서 정보를 추출하고 추론하는 LLM의 능력을 향상시켜 더 정확하고 유익한 응답을 생성할 수 있음을 보여줍니다.


**참고:**  
이 노트북에서는 NVIDIA API 카탈로그에서 제공하는 NVIDIA NIM 마이크로서비스를 사용했습니다.  
위의 API들, `NVIDIA (llms)`, `NVIDIAEmbedding`, 그리고 [Azure AI Search Semantic Hybrid Retrieval (내장된 재정렬 기능)](https://learn.microsoft.com/azure/search/semantic-search-overview)을 포함합니다. 참고로, 위의 API들은 자체 호스팅 마이크로서비스도 지원할 수 있습니다.  

**예제:**  
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")
```



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있지만, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 신뢰할 수 있는 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
